### Problem Description 

* population = set of n individuals 
* an individual = one valid fantasy league
* Each league consists of 5 teams
* Each team is itself a list of 7 player dictionaries

<br> List of Individuals → Each Individual = List of 5 Teams → Each Team = List of 7 Players → Each Player = Dictionary

 Constraints:
 * every player must be assigned to exactly one team
 * each of the 5 teams must have 1 GK, 2 DEF, 2 MID, 2 FWD
 * the total salary of any team cannot exceed €750 million
 * all teams must be formed (no incompete teams nor not assigned players) 

In [4]:
import random
from copy import deepcopy
import pandas as pd

In [5]:
def load_players_from_csv(filepath):

    df = pd.read_csv(filepath)
    
    #creating a list of players
    players = []
    for _, row in df.iterrows():
        player = {
            "name": row['Name'],
            "position": row['Position'],
            "skill":int(row['Skill']),
            "cost": float(row['Salary (€M)'])
        }
        players.append(player)
    return players

We have to decide later what population size we want to set. By default i set 50.

In [9]:
def create_initial_population(players, population_size=50, salary_cap=750):
    
    population = []

    GKs = [p for p in players if p['position'] == 'GK']
    DEFs = [p for p in players if p['position'] == 'DEF']
    MIDs = [p for p in players if p['position'] == 'MID']
    FWDs = [p for p in players if p['position'] == 'FWD']

    for _ in range(population_size):
        valid = False
        while not valid:
            used_player_names = set()
            teams = []
            random.shuffle(GKs)
            random.shuffle(DEFs)
            random.shuffle(MIDs)
            random.shuffle(FWDs)

            for _ in range(5):
                
                team_gk = [p for p in GKs if p['name'] not in used_player_names][:1]
                team_defs = [p for p in DEFs if p['name'] not in used_player_names][:2]
                team_mids = [p for p in MIDs if p['name'] not in used_player_names][:2]
                team_fwds = [p for p in FWDs if p['name'] not in used_player_names][:2]
                
                team = team_gk + team_defs + team_mids + team_fwds
                
                
                if len(team) != 7:
                    break 

                for player in team:
                    used_player_names.add(player['name'])
                
                teams.append(team)

            if len(teams) == 5:
                over_budget = False
                for team in teams:
                    team_cost = sum(p['cost'] for p in team)
                    if team_cost > salary_cap:
                        over_budget = True
                        break
                
                if not over_budget:
                    valid = True
                    population.append(deepcopy(teams))

    return population


In [18]:
players = load_players_from_csv("data/players(in).csv")
players

[{'name': 'Alex Carter', 'position': 'GK', 'skill': 85, 'cost': 90.0},
 {'name': 'Jordan Smith', 'position': 'GK', 'skill': 88, 'cost': 100.0},
 {'name': 'Ryan Mitchell', 'position': 'GK', 'skill': 83, 'cost': 85.0},
 {'name': 'Chris Thompson', 'position': 'GK', 'skill': 80, 'cost': 80.0},
 {'name': 'Blake Henderson', 'position': 'GK', 'skill': 87, 'cost': 95.0},
 {'name': 'Daniel Foster', 'position': 'DEF', 'skill': 90, 'cost': 110.0},
 {'name': 'Lucas Bennett', 'position': 'DEF', 'skill': 85, 'cost': 90.0},
 {'name': 'Owen Parker', 'position': 'DEF', 'skill': 88, 'cost': 100.0},
 {'name': 'Ethan Howard', 'position': 'DEF', 'skill': 80, 'cost': 70.0},
 {'name': 'Mason Reed', 'position': 'DEF', 'skill': 82, 'cost': 75.0},
 {'name': 'Logan Brooks', 'position': 'DEF', 'skill': 86, 'cost': 95.0},
 {'name': 'Caleb Fisher', 'position': 'DEF', 'skill': 84, 'cost': 85.0},
 {'name': 'Nathan Wright', 'position': 'MID', 'skill': 92, 'cost': 120.0},
 {'name': 'Connor Hayes', 'position': 'MID', 's

In [17]:
initial_population = create_initial_population(players,population_size=50, salary_cap=750)
initial_population

[[[{'name': 'Ryan Mitchell', 'position': 'GK', 'skill': 83, 'cost': 85.0},
   {'name': 'Lucas Bennett', 'position': 'DEF', 'skill': 85, 'cost': 90.0},
   {'name': 'Mason Reed', 'position': 'DEF', 'skill': 82, 'cost': 75.0},
   {'name': 'Hunter Cooper', 'position': 'MID', 'skill': 83, 'cost': 85.0},
   {'name': 'Dylan Morgan', 'position': 'MID', 'skill': 91, 'cost': 115.0},
   {'name': 'Chase Murphy', 'position': 'FWD', 'skill': 86, 'cost': 95.0},
   {'name': 'Sebastian Perry', 'position': 'FWD', 'skill': 95, 'cost': 150.0}],
  [{'name': 'Chris Thompson', 'position': 'GK', 'skill': 80, 'cost': 80.0},
   {'name': 'Caleb Fisher', 'position': 'DEF', 'skill': 84, 'cost': 85.0},
   {'name': 'Owen Parker', 'position': 'DEF', 'skill': 88, 'cost': 100.0},
   {'name': 'Nathan Wright', 'position': 'MID', 'skill': 92, 'cost': 120.0},
   {'name': 'Ashton Phillips', 'position': 'MID', 'skill': 90, 'cost': 110.0},
   {'name': 'Julian Scott', 'position': 'FWD', 'skill': 92, 'cost': 130.0},
   {'name':

In [27]:
for team_idx, team in enumerate(initial_population[0], start=1):
    print(f"\nTeam {team_idx}")
    for player in team:
        print(f"{player['name']}, {player['position']}, Skill: {player['skill']}, Salary: €{player['cost']}M")
        
    team_cost = sum(player['cost'] for player in team)
    print(f"Total Team Salary: €{team_cost}M")



Team 1
Ryan Mitchell, GK, Skill: 83, Salary: €85.0M
Lucas Bennett, DEF, Skill: 85, Salary: €90.0M
Mason Reed, DEF, Skill: 82, Salary: €75.0M
Hunter Cooper, MID, Skill: 83, Salary: €85.0M
Dylan Morgan, MID, Skill: 91, Salary: €115.0M
Chase Murphy, FWD, Skill: 86, Salary: €95.0M
Sebastian Perry, FWD, Skill: 95, Salary: €150.0M
Total Team Salary: €695.0M

Team 2
Chris Thompson, GK, Skill: 80, Salary: €80.0M
Caleb Fisher, DEF, Skill: 84, Salary: €85.0M
Owen Parker, DEF, Skill: 88, Salary: €100.0M
Nathan Wright, MID, Skill: 92, Salary: €120.0M
Ashton Phillips, MID, Skill: 90, Salary: €110.0M
Julian Scott, FWD, Skill: 92, Salary: €130.0M
Xavier Bryant, FWD, Skill: 90, Salary: €120.0M
Total Team Salary: €745.0M

Team 3
Blake Henderson, GK, Skill: 87, Salary: €95.0M
Ethan Howard, DEF, Skill: 80, Salary: €70.0M
Maxwell Flores, DEF, Skill: 81, Salary: €72.0M
Austin Torres, MID, Skill: 82, Salary: €80.0M
Dominic Bell, MID, Skill: 86, Salary: €95.0M
Tyler Jenkins, FWD, Skill: 80, Salary: €70.0M
Z